Pydantic モデルの定義ファイルを自動生成できるようにする

In [1]:
from typing import NotRequired, TypedDict

from pydantic import ConfigDict


class TypeSpec(TypedDict):
    name: str
    import_from: NotRequired[str]
    alias: NotRequired[str]
    list: NotRequired[bool]
    optional: NotRequired[bool]


class ModelSpec(TypedDict):
    name: str
    description: NotRequired[str]
    fields: list[tuple[str, TypeSpec]]
    base: NotRequired[TypeSpec]
    config: NotRequired[ConfigDict]

In [2]:
from collections import defaultdict


def build_definitions(models: list[ModelSpec]) -> str:
    def type_ref(t: TypeSpec) -> str:
        ref = t.get("alias", t["name"])
        if t.get("list"):
            ref = f"list[{ref}]"
        if t.get("optional"):
            ref = f"{ref} | None"
        return ref

    # Collect imports: module -> {(name, alias)}
    imports: dict[str, set[tuple[str, str | None]]] = defaultdict(set)

    for m in models:
        # base type import (e.g., BaseModel from pydantic)
        base = m.get("base")
        if base and base.get("import_from"):
            imports[base["import_from"]].add((base["name"], base.get("alias")))

        # field type imports
        for field_name, field_type in m["fields"]:
            if field_type.get("import_from"):
                imports[field_type["import_from"]].add(
                    (field_type["name"], field_type.get("alias"))
                )

    # Render imports in a deterministic order
    import_lines: list[str] = []
    for module in sorted(imports.keys()):
        items = sorted(imports[module], key=lambda x: (x[0], x[1] or ""))
        parts = []
        for name, alias in items:
            parts.append(f"{name} as {alias}" if alias else name)
        import_lines.append(f"from {module} import {', '.join(parts)}")

    # Render classes
    class_lines: list[str] = []
    for m in models:
        base = m.get("base")
        base_name = type_ref(base) if base else "BaseModel"

        # Add BaseModel import if base is not specified
        if not base:
            imports.setdefault("pydantic", set()).add(("BaseModel", None))

        class_lines.append(f"class {m['name']}({base_name}):")

        # Add description as docstring if present
        description = m.get("description")
        if description:
            class_lines.append(f'    """{description}"""')
            class_lines.append("")

        # Add config if present
        config = m.get("config")
        if config:
            # Format config as ConfigDict(key1=value1, key2=value2, ...)
            config_items = []
            for key, value in config.items():
                if isinstance(value, str):
                    config_items.append(f"{key}='{value}'")
                else:
                    config_items.append(f"{key}={value!r}")
            config_str = ", ".join(config_items)
            class_lines.append(f"    model_config = ConfigDict({config_str})")
            class_lines.append("")
            # Add ConfigDict import
            imports.setdefault("pydantic", set()).add(("ConfigDict", None))

        # Add fields
        if m["fields"]:
            for field_name, field_type in m["fields"]:
                class_lines.append(f"    {field_name}: {type_ref(field_type)}")
        else:
            if not config and not description:
                class_lines.append("    pass")

        class_lines.append("")  # blank line after each class

    # Re-render imports if ConfigDict or BaseModel was added
    import_lines = []
    for module in sorted(imports.keys()):
        items = sorted(imports[module], key=lambda x: (x[0], x[1] or ""))
        parts = []
        for name, alias in items:
            parts.append(f"{name} as {alias}" if alias else name)
        import_lines.append(f"from {module} import {', '.join(parts)}")

    # Join (imports, blank line, classes). Match the sample formatting.
    out: list[str] = []
    out.extend(import_lines)
    if import_lines:
        out.append("")  # blank line between imports and first class
    out.extend(class_lines)

    # Avoid extra blank lines at the very end (sample ends without an extra blank line)
    while out and out[-1] == "":
        out.pop()

    return "\n".join(out)

In [3]:
User: ModelSpec = {
    "name": "User",
    "description": "Represents a user in the system",
    "fields": [
        ("id", {"name": "UUID", "import_from": "uuid"}),
        ("name", {"name": "str"}),
        ("meta", {"name": "UserMeta", "import_from": "models"}),
        # relations
        ("posts", {"name": "Post", "list": True, "optional": True}),
    ],
}

Post: ModelSpec = {
    "name": "Post",
    "description": "Represents a blog post written by a user",
    "fields": [
        ("id", {"name": "UUID", "import_from": "uuid"}),
        ("title", {"name": "str"}),
        ("content", {"name": "str"}),
        ("created_at", {"name": "datetime", "import_from": "datetime"}),
        ("updated_at", {"name": "datetime", "import_from": "datetime"}),
        # relations
        ("author", {"name": "User", "optional": True}),
    ],
    "config": {"frozen": True, "extra": "forbid"},
}

print(build_definitions([User, Post]))

from datetime import datetime
from models import UserMeta
from pydantic import BaseModel, ConfigDict
from uuid import UUID

class User(BaseModel):
    """Represents a user in the system"""

    id: UUID
    name: str
    meta: UserMeta
    posts: list[Post] | None

class Post(BaseModel):
    """Represents a blog post written by a user"""

    model_config = ConfigDict(frozen=True, extra='forbid')

    id: UUID
    title: str
    content: str
    created_at: datetime
    updated_at: datetime
    author: User | None
